# Serving the model locally

In [ ]:
# To serve the model locally, go to the anaconda prompt and run the following command:

# (base) C:\Users\your_user>mlflow models serve --model-uri runs:/ID of the best model in Mlflow/model_name -p 2345

# After that, the model will be ready to be consumed in production via API.

# Importing and reading the data

In [3]:
# The 'requests' package is used to make HTTP requests, such as sending data to an API or consuming a model prediction in production
import requests

# The 'pandas' package is used for data manipulation and analysis in table format (DataFrames), facilitating processing and visualization of results via API.
import pandas as pd


In [6]:
credit_data = pd.read_csv("Credit.csv")

In [5]:
credit_data.head(5) # Shows only the first 5 rows of the dataset

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,'critical/other existing credit',radio/tv,1169,'no known savings',>=7,4,'male single',none,...,'real estate',67,none,own,2,skilled,1,yes,yes,good
1,0<=X<200,48,'existing paid',radio/tv,5951,<100,1<=X<4,2,'female div/dep/mar',none,...,'real estate',22,none,own,1,skilled,1,none,yes,bad
2,'no checking',12,'critical/other existing credit',education,2096,<100,4<=X<7,2,'male single',none,...,'real estate',49,none,own,1,'unskilled resident',2,none,yes,good
3,<0,42,'existing paid',furniture/equipment,7882,<100,4<=X<7,2,'male single',guarantor,...,'life insurance',45,none,'for free',1,skilled,2,none,yes,good
4,<0,24,'delayed previously','new car',4870,<100,1<=X<4,3,'male single',none,...,'no known property',53,none,'for free',2,skilled,2,none,yes,bad


# Converting string variables to numerical variables

In [7]:
# Iterates through all columns of the 'credit_data' DataFrame
for col in credit_data.columns:
    # Checks if the column type is 'object' (usually text/categorical)
    if credit_data[col].dtype == 'object':
        # Converts the column to the 'category' type and then transforms the categories into numerical codes
        credit_data[col] = credit_data[col].astype('category').cat.codes


In [8]:
credit_data

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,2,6,1,7,1169,0,3,4,3,2,...,2,67,1,1,2,3,1,1,1,1
1,1,48,3,7,5951,3,0,2,0,2,...,2,22,1,1,1,3,1,0,1,0
2,0,12,1,4,2096,3,1,2,3,2,...,2,49,1,1,1,2,2,0,1,1
3,2,42,3,5,7882,3,1,2,3,1,...,0,45,1,0,1,3,2,0,1,1
4,2,24,2,1,4870,3,0,3,3,2,...,1,53,1,0,2,3,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,12,3,5,1736,3,1,3,0,2,...,2,31,1,1,1,2,1,0,1,1
996,2,30,3,2,3857,3,0,4,1,2,...,0,40,1,1,1,0,1,1,1,1
997,0,12,3,7,804,3,3,4,3,2,...,3,38,1,1,1,3,1,0,1,1
998,2,45,3,7,1845,3,0,4,3,2,...,1,23,1,0,1,3,1,1,1,0


# Sending the request to consume the model and make predictions

In [10]:
import requests
import socket
import json

def check_port_open(host, port):
    """Test if a port is open on the given host"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)  # 2 second timeout
    result = sock.connect_ex((host, port))
    sock.close()
    return result == 0  # Returns True if port is open

# First check if anything is listening on port 2345
if check_port_open('localhost', 2345):
    print("Port 2345 is open! Attempting to connect to the model server...")
    
    # Try a simple GET request first to test basic connectivity
    try:
        response = requests.get('http://localhost:2345/', timeout=5)
        print(f"GET request status: {response.status_code}")
        print(f"Response: {response.text[:100]}...")  # Print first 100 chars
    except requests.exceptions.RequestException as e:
        print(f"GET request failed: {e}")
    
    # Then try your actual POST request
    try:
        prediction = requests.post(
            url='http://localhost:2345/invocations',
            headers={'Content-Type': 'application/json'},
            data=json.dumps({
                "inputs": [[1.0, 2.0, 3.0]]  # Simple test data
            }),
            timeout=5
        )
        print(f"POST request status: {prediction.status_code}")
        print(f"Response: {prediction.text[:100]}...")
    except requests.exceptions.RequestException as e:
        print(f"POST request failed: {e}")
else:
    print("Port 2345 is not open. The model server is not running or is using a different port.")
    
    # Try some common alternative ports for model serving
    alternative_ports = [8080, 8000, 5000, 8501, 8502]
    print("Checking alternative ports...")
    
    for port in alternative_ports:
        if check_port_open('localhost', port):
            print(f"Found open port at {port}. Try using this port instead.")

Port 2345 is not open. The model server is not running or is using a different port.
Checking alternative ports...


In [ ]:
prediction

In [ ]:
prediction.text